notebook to perform SMOTE oversampling in Python so that the classes are approximately equal

# Data Cleaning
first, clean the data

In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.impute import KNNImputer

I had to mount my drive here to get this to work

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_excel("/content/drive/MyDrive/01-Classes/08Semester/CSDS491/CSDS491CourseProject/data/UHsurv1500_withZeroFE.xlsx") #Sakin's directory
#data = pd.read_excel("/content/drive/MyDrive/Case/Graph Class/UHsurv1500_withZeroFE.xlsx") #Josh's directory
#data2 = pd.read_excel("/content/drive/MyDrive/Case/Graph Class/clarify_1500 _withZero.xlsx") #Josh's directory
#data['ID']=data2['ID']
#data=data.merge(data2,how='outer', on='ID',suffixes=('_left', '_right'))#,lsuffix='ca',rsuffix='pa')


In [ ]:
data.shape

(2457, 80)

In [ ]:
data.head()

,AgatstonScore2D,MassScore,VolumeScore,Area2D,NumLesion3D,isAgZero,AgatstonScore3D,isLesion3DBelow5,Age,isAgeBelow60,...,ICmeanMomentH2,ICmeanMomentH3,ICskewnesstMomentH1,ICskewnesstMomentH2,ICskewnesstMomentH3,ICkurtosisMomentH1,ICkurtosisMomentH2,ICkurtosisMomentH3,Time,Status
0,38.146973,5.324782,37.765503,15.106201,1,False,50.354004,True,61,False,...,5.711942,3.557781,2.565725,2.287119,0.974375,2.953309,3.076700,2.858016,104,0
1,74.160576,11.911143,82.431793,32.972717,2,False,104.862213,True,65,False,...,5.013292,2.922836,3.337986,2.402399,0.870613,3.277002,3.248232,2.911082,642,0
2,88.413556,13.951670,81.658363,32.663345,3,False,95.963478,True,84,False,...,21.370782,5.834826,34.207528,35.952442,5.134972,2.652832,2.642296,2.605456,635,1
3,716.765938,0.982489,9.476852,230.224335,2,False,5.370216,True,80,False,...,3.677168,1.844542,0.136258,0.177513,0.054845,1.915504,1.901077,1.904156,300,1
4,446.760742,69.292085,517.397461,172.795898,14,False,492.311523,False,58,True,...,13.813606,5.981036,21.602994,31.398427,8.830517,3.733239,3.725690,3.694552,1923,0


### Removing irrelevant data
the data was taken directly from a dataset that we want to investigate, so none of the features will be removed. check the columns to make sure some columns arent duplicated or anything

In [ ]:
data=data.drop(['AgatstonScore3D','AgatstonScore2D','isAgeBelow60','isFemale', 'HUperArtery2D_stat_LM2','HUperArtery2D_stat_LM3','HUperArtery2D_stat_LM4'
,'HUperArtery2D_stat_LAD2','HUperArtery2D_stat_LAD3','HUperArtery2D_stat_LAD4'
,'HUperArtery2D_stat_LCX2','HUperArtery2D_stat_LCX3'
,'HUperArtery2D_stat_LCX4','HUperArtery2D_stat_RCA2'
,'HUperArtery2D_stat_RCA3','HUperArtery2D_stat_RCA4'],axis=1)

#data=data.drop(['anonExamDate','anonPatName','anonPatMRN','anonPatDoB','anonAccNo', 'AnonPath','Dir','FullCTfolder'
#,'CTfolder','Maskfolder','ID'
#,'Selected','MACE_after_days','MACE_after','STRESS_CTA_365_days','STRESS_CTA_365','CORANGIO_365_days'
#,'CORANGIO_365','CABG_PCI','CABG_PCI_days','DM_DAYS','CABG_PCI_365','CABG_PCI_days_365','MACEr_AFTER_DAYS_CENS365'
#,'MACEr_AFTER_CENS365','STROKE_AFTER','STROKE_AFTER_DAYS','MACEr_AFTER','MACEr_AFTER_DAYS','MI_AFTER','MI_AFTER_DAYS'
#,'HF_AFTER','HF_AFTER_DAYS','days_death','DIED','PCI','PCI_DAYS','CORANGIO','CORANGIO_DAYS','CABG','CABG_DAYS'
#,'any_statin_days','any_statin','CCTA_days','CCTA','stress_days','stress_testing','aspirin_before','hi_statin_before'
#,'statin_before','PCE10yrrisk','BASELINE_ANTIHTN','Race_Grp',
#       'Baseline.DBP', 'Baseline.SBP', 'Baseline_result.A1C','Race_Grp',
#       'Baseline_result.HDL', 'Baseline_result.LDL', 'Baseline_result.TC',
#       'Baseline_result.TG','LDL_MAX'],axis=1)
data.columns

KeyError: ignored

### data deduplication 
duplicated data can give more weight to a model that is trained on the data, so we should remove the duplicated data.

In [ ]:
working = data.drop_duplicates()
working.shape

(2448, 64)

So, we had 7 duplicated patients. dropping them will improve our model

### missing data


In [ ]:
null_vals = data.isnull().sum()
print(null_vals.values)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


no data is missing, so we can move to SMOTE

In [ ]:
imputer = KNNImputer(n_neighbors=2)
data_nimp=data
names=data.columns
temp_data = imputer.fit_transform(data_nimp,data_nimp['Baseline.BMI'])

KeyError: ignored

In [ ]:
data=pd.DataFrame(temp_data,columns=names)

NameError: ignored

In [ ]:
null_vals = data.isnull().sum()
print(null_vals.values)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
data

,MassScore,VolumeScore,Area2D,NumLesion3D,isAgZero,isLesion3DBelow5,Age,AgGroupX1,AgGroupX2,AgGroupX3,...,ICmeanMomentH2,ICmeanMomentH3,ICskewnesstMomentH1,ICskewnesstMomentH2,ICskewnesstMomentH3,ICkurtosisMomentH1,ICkurtosisMomentH2,ICkurtosisMomentH3,Time,Status
0,5.324782,37.765503,15.106201,1,False,True,61,True,False,False,...,5.711942,3.557781,2.565725,2.287119,0.974375,2.953309,3.076700,2.858016,104,0
1,11.911143,82.431793,32.972717,2,False,True,65,True,False,False,...,5.013292,2.922836,3.337986,2.402399,0.870613,3.277002,3.248232,2.911082,642,0
2,13.951670,81.658363,32.663345,3,False,True,84,True,False,False,...,21.370782,5.834826,34.207528,35.952442,5.134972,2.652832,2.642296,2.605456,635,1
3,0.982489,9.476852,230.224335,2,False,True,80,False,False,True,...,3.677168,1.844542,0.136258,0.177513,0.054845,1.915504,1.901077,1.904156,300,1
4,69.292085,517.397461,172.795898,14,False,False,58,False,False,True,...,13.813606,5.981036,21.602994,31.398427,8.830517,3.733239,3.725690,3.694552,1923,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452,0.000000,0.000000,0.000000,0,True,True,57,False,False,False,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,982,0
2453,0.000000,0.000000,0.000000,0,True,True,46,False,False,False,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,458,0
2454,0.000000,0.000000,0.000000,0,True,True,39,False,False,False,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,39,0
2455,0.000000,0.000000,0.000000,0,True,True,72,False,False,False,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,835,0


# SMOTE
fit the data and upsample the smaller class using SMOTE

In [ ]:
X = data.loc[:, data.columns != "Status"]
Y = data.loc[:, data.columns == "Status"]

In [ ]:
# initialize SMOTE
os = SMOTE(random_state=1234)

# train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1234)

double-check that the status column is not included in the train/test data

In [ ]:
X_train.columns

Index(['MassScore', 'VolumeScore', 'Area2D', 'NumLesion3D', 'isAgZero',
       'isLesion3DBelow5', 'Age', 'AgGroupX1', 'AgGroupX2', 'AgGroupX3',
       'numLesionPerArtery3D_LM1', 'numLesionPerArtery3D_LAD1',
       'numLesionPerArtery3D_LCX1', 'numLesionPerArtery3D_RCA1', 'isArt2plus',
       'isArt3plus', 'numArtCalc', 'AgastonScorePerArtery2D_LM1',
       'AgastonScorePerArtery2D_LAD1', 'AgastonScorePerArtery2D_LCX1',
       'AgastonScorePerArtery2D_RCA1', 'MassScorePerArtery_LM1',
       'MassScorePerArtery_LAD1', 'MassScorePerArtery_LCX1',
       'MassScorePerArtery_RCA1', 'VolumeScorePerArtery_LM1',
       'VolumeScorePerArtery_LAD1', 'VolumeScorePerArtery_LCX1',
       'VolumeScorePerArtery_RCA1', 'massHist1', 'massHist2', 'massHist3',
       'massHist4', 'massHist5', 'avrHist1', 'avrHist2', 'avrHist3',
       'avrHist4', 'avrHist5', 'DistTop2LastLesionPerArtery_LM1',
       'DistTop2LastLesionPerArtery_LAD1', 'DistTop2LastLesionPerArtery_LCX1',
       'DistTop2LastLesionPerArte

fit with SMOTE

In [ ]:
os_data_X, os_data_Y = os.fit_resample(X_train, Y_train)
X_train_os = pd.DataFrame(data=os_data_X)
Y_train_os = pd.DataFrame(data=os_data_Y)

compare with the original proportions

In [ ]:
Y_train.describe()

,Status
count,1965.000000
mean,0.143511
std,0.350683
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [ ]:
Y_train_os.describe()

,Status
count,3366.000000
mean,0.500000
std,0.500074
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


the mean shows us the proportion of 0s and 1s. In the original dataset, it is split 14$ 1 and 86% 0s. In the new upsampled dataset, we have 50% 1s and 50% 0s.

In [ ]:
# convert to DataFrame and write to csv files (test and train data)
train_data = pd.concat((X_train_os, Y_train_os), axis=1, ignore_index=False)
test_data = pd.concat((X_test, Y_test), axis=1, ignore_index=False)

In [ ]:
train_data.head()

,MassScore,VolumeScore,Area2D,NumLesion3D,isAgZero,isLesion3DBelow5,AgGroupX1,AgGroupX2,AgGroupX3,numLesionPerArtery3D_LM1,...,female,smokingstatus,Baseline.BMI,IBD,SLE,Psoriasis,RA,BASELINE_DM,DM,Status
0,0.000000,0.000000,0.000000,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,33.43,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11.391991,95.593929,38.623810,12.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,36.50,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2,0.000000,0.000000,0.000000,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,30.35,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,21.32,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14.425980,76.675415,30.975342,3.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,1.0,33.76,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
test_data.head()

,MassScore,VolumeScore,Area2D,NumLesion3D,isAgZero,isLesion3DBelow5,AgGroupX1,AgGroupX2,AgGroupX3,numLesionPerArtery3D_LM1,...,female,smokingstatus,Baseline.BMI,IBD,SLE,Psoriasis,RA,BASELINE_DM,DM,Status
1224,10.236033,62.452927,24.981171,4.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,29.840,0.0,0.0,0.0,0.0,0.0,0.0,0.0
263,122.015518,755.190849,303.983688,30.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,31.940,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1580,0.000000,0.000000,0.000000,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,29.935,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1315,6.859407,37.911892,15.164757,2.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,21.620,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1169,36.141647,208.282471,83.312988,5.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,25.557,0.0,0.0,0.0,0.0,0.0,0.0,0.0


write to csv to save the upsampled data so that the various models we are training are trained using consistent training and testing data

In [ ]:
#train_data.to_csv("/content/drive/MyDrive/01-Classes/08Semester/CSDS491/CSDS491CourseProject/data/train.csv") #Sakin's directory
train_data.to_csv("/content/drive/MyDrive/Case/Graph Class/train.csv") #Josh's directory
#test_data.to_csv("/content/drive/MyDrive/01-Classes/08Semester/CSDS491/CSDS491CourseProject/data/test.csv") #Sakin's directory
test_data.to_csv("/content/drive/MyDrive/Case/Graph Class/test.csv") #Josh's directory